In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Introducción

En esta clase vamos a comparar las performance de los distintos modelos basados en ensambles de árboles:

* Árbol de ensamble -> Bagging
* Árbol de ensamble -> Random Forest


## Dataset

En esta clase usaremos un dataset con info de películas ("Movie_classification.csv").  
Este dataset esta conformado por los siguientes features:  

 *   **Marketing expense:**    (float64)    Gasto total en Marketing      
 *   **Production expense:**   (float64)    Gasto total de Producción
 *   **Multiplex coverage:**   (float64)    Cobertura promedio de Multiplex
 *   **Budget:**               (float64)    Presupuesto
 *   **Movie_length:**         (float64)    Duración de la película
 *   **Lead_ Actor_Rating:**   (float64)    Puntaje sobre el actor principal
 *   **Lead_Actress_rating:**  (float64)    Puntaje sobre la actriz principal
 *   **Director_rating:**      (float64)    Puntaje sobre el Director
 *   **Producer_rating:**      (float64)    Puntaje sobre el Productor
 *   **Critic_rating:**        (float64)    Puntaje que le puso la crítica
 *   **Trailer_views:**        (int64)      Cantidad de vistas del Trailer
 *   **3D_available:**         (object)     Si esta disponible en 3D (Yes/No)
 *   **Time_taken:**           (float64)    Duración de la película
 *   **Twitter_hastags:**      (float64)    Cantidad de menciones en twitter
 *   **Genre:**                (object)     Genero de la película
 *   **Avg_age_actors:**       (int64)      Edad promedio de los actores
 *   **Num_multiplex:**        (int64)      Cantidad de Multiplex
 *   **Collection:**           (int64)      Recaudación
 *   **Start_Tech_Oscar:**     (int64)      Si recibió un oscar o no.
 
 


## Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Ejercicio 1 - Importar datos

1) Leamos los datos de "Movie_classification.csv" y lo guardamos en un dataframe de pandas.  
2) Veamos cuántos registros hay en cada DataFrame y de qué tipos son los datos de cada columna.   
3) Veamos los primeros registros de cada DataFrame para verificar que los datos fueron importados correctamente.

In [ ]:
df = pd.read_csv("../Data/Movie_classification.csv", header=0)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head()

## Ejercicio 2 - Imputación de valores faltantes

Veamos si existen valores faltantes y en tal caso imputemos los mismos.

In [ ]:
#vemos la cantidad de valores que tiene cada columna
df.info()

In [ ]:
# observamos que time_taken es la unica columna que no tiene 506 observaciónes, 
# por ende vamos a imputar los valores faltantes utilizando la media.
df['Time_taken'].mean()

In [ ]:
df['Time_taken'].fillna(value = df['Time_taken'].mean(), inplace = True)

In [ ]:
df.info()

## Ejercicio 3 - Generación de Variables Dummies.

Veamos si existen variables categóricas y en tal caso generar variables dummies para dichas columnas.

In [ ]:
df.dtypes.loc[df.dtypes=="object"]  

In [ ]:
df[['3D_available','Genre']].head()

In [ ]:
df = pd.get_dummies(df,columns = ["3D_available","Genre"],drop_first = True)

In [ ]:
df.head()

## Ejercicio 4 - Features, Target

Construyamos una matriz de features (X) y el vector target (Y) para predecir `Start_Tech_Oscar` en el dataset de datos completos

¿Qué valores toma la variable `Start_Tech_Oscar` en el dataset?


In [ ]:
X = df.loc[:,df.columns!="Start_Tech_Oscar"]
type(X)

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
y = df["Start_Tech_Oscar"]
type(y)

In [ ]:
y.head()

In [ ]:
y.shape

## Ejercicio 5 - Train Test Split

Constuyamos los conjuntos de train y test, asignando el 70% de los registros a train y el 30% a test

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=0)

In [ ]:
X_train.head()

In [ ]:
X_train.shape

In [ ]:
X_test.shape

## Ejercicio 6 - Construimos un modelo utilizando Bagging

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html  

1) creamos un clasificador de árbol simple.  
2) con este clasificador simple, generar el meta-modelo basado en la técnica de Bagging (utilizar 1000 estimadores).  
3) entrenamos el modelo de Bagging.  
4) calculamos la matriz de confusión  
5) calculamos el accuracy tanto para el dataset de prueba.  

In [ ]:
from sklearn import tree
clftree = tree.DecisionTreeClassifier()

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
bag_clf = BaggingClassifier(base_estimator=clftree, n_estimators=1000,
                            bootstrap=True, n_jobs=-1,
                            random_state=42)

In [ ]:
bag_clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
confusion_matrix(y_test, bag_clf.predict(X_test))

In [ ]:
accuracy_score(y_test, bag_clf.predict(X_test))

## Ejercicio 7 - Construimos un modelo utilizando Random Forest

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

Algunos de los parámetros más importantes son los siguientes:

* `n_estimators`: el número de iteraciones (o sea, de `base_estimators`) para entrenar
* `criterion`: define el criterio de impureza para evaluar la calidad de las particiones (por defecto, es `gini`) 
* `max_features`: la cantidad de features que extraerá para entrenar cada `base_estimator`. Por default es igual a `sqrt(X.shape[1])`
* `bootstrap` y `bootstrap_features`: controla si tanto los n_samples como las features son extraidos con reposición.
* `max_depth`: la pronfundidad máxima del árbol
* `min_samples_leaf`: el número mínimo de n_samples para constituir una hoja del árbol (nodo terminal)
* `min_samples_split`: el número mínimo de n_samples para realizar un split.

y varios otros que pueden llegar a ser importantes al momento de realizar el tunning. En general, los más importantes suelen ser: `n_estimators`, `max_features`, `max_depth` y `min_samples_leaf`.
  


### Ejercicio
1) Generaramos el meta-modelo basado en la técnica de Random Forest (utilizar 1000 estimadores)  
2) entrenamos el modelo   
3) calculamos la matriz de confusión    
4) calculamos el accuracy tanto para el dataset de prueba.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1 ,random_state=42)

In [ ]:
rf_clf.fit(X_train, y_train)

In [ ]:
confusion_matrix(y_test, rf_clf.predict(X_test))

In [ ]:
accuracy_score(y_test, rf_clf.predict(X_test))

## Ejercicio 8 - Construimos un modelo utilizando ExtraTreesClassifier()

Este modelo es una variación del Random Forest que busca ser más rápido. Esto lo hace evitando buscar el punto de split optimo en cada nodo de los árboles, sino que por el contrario, selecciona un split de forma aleatoria. Esto hace que el modelo se vuelva más veloz, pudiendo alcanzar o incluso superar la precisión alcanzada por Random Forest.

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html  
https://www.youtube.com/watch?v=Q1qpG7gwix4

### Ejercicio
1) Generaramos el meta-modelo basado en la técnica de ExtraTrees (utilizar 10000 estimadores)  
2) entrenamos el modelo   
3) calculamos la matriz de confusión    
4) calculamos el accuracy tanto para el dataset de prueba.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
et = ExtraTreesClassifier(n_estimators=10000, class_weight='balanced', random_state=1)

In [ ]:
et.fit(X_train, y_train)

In [ ]:
confusion_matrix(y_test, et.predict(X_test))

In [ ]:
accuracy_score(y_test, et.predict(X_test))